In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
from datasets import Dataset

# Custom dataset in the desired format
data ={
    "instruction": [
        "Identify the type of document and extract named entities such as name, email, ID, age, address, and others, if present."
    ] * 10,
    "input": [
        "adhar- Government of India Rahul Rajendra prasad Mishra DOB: 25/05/1985 Male 2932 1448 0395 VID: 9155 9158 0007 4305 Issue Date: 25/05/2019",
        "Jane Doe 123 Great Linclon Road Miami MA 11223 Check No. Date 26/06/2024 Pay to the order of Cairo Mohammed $2200 The sum of two thousand two hundred dollars Miami City Saving Union Miami Centre Branch Highland and Avenue 123 Miami USA Memo Authorized signature A1234 – 556789839292892 – 0123",
        "Income tax department Govt of India Permanent Account Number Card LIWPS9203C Name Amardeep Singh Father’s Name: Surjit Singh Date of Birth – 09/11/1995 Signature",
        "American Express 3159 876543 21001 valid till 10/28 CF Frost Amea",
        "Aadhaar- Government of India Aarav Kapoor DOB: 14/02/1990 Male 7894 5623 1478 VID: 1234 5678 9012 3456 Issue Date: 14/02/2020",
        "Neha Sharma 45 Elite Avenue Pune MH 411045 Check No. Date 30/09/2023 Pay to the order of Amardeep Singh $1500 The sum of one thousand five hundred dollars Pune City Cooperative Bank Pune Elite Branch Memo Authorized signature B5678 – 556789839292892 – 0123",
        "Income tax department Govt of India Permanent Account Number Card HGXPS3452K Name Jane Doe Father’s Name: Michael Doe Date of Birth – 12/08/1982 Signature",
        "MasterCard 5678 4321 2109 8765 valid till 07/25 Neha Sharma USA",
        "Michael Doe 1234 Main St, New York, NY, 10001 Check No. Date 01/12/2023 Pay to the order of Aarav Kapoor $5000 The sum of five thousand dollars NY State Bank NYC Branch Memo Authorized signature C9012 – 556789839292892 – 0123",
        "Aadhaar- Government of India CF Frost DOB: 1980 Male 6543 3210 9087 VID: 2109 8765 4321 1098 Issue Date: 1985"
    ],
    "output": [
        "{\n  \"document_type\": \"aadhaar\",\n  \"named_entities\": {\n    \"name\": \"Rahul Rajendra prasad Mishra\",\n    \"dob\": \"25/05/1985\",\n    \"gender\": \"Male\",\n    \"aadhaar_number\": \"2932 1448 0395\",\n    \"vid\": \"9155 9158 0007 4305\",\n    \"issue_date\": \"25/05/2019\"\n  }\n}",
        "{\n  \"document_type\": \"cheque\",\n  \"named_entities\": {\n    \"name\": \"Jane Doe\",\n    \"address\": \"123 Great Linclon Road Miami MA 11223\",\n    \"cheque_number\": \"A1234\",\n    \"date\": \"26/06/2024\",\n    \"payee\": \"Cairo Mohammed\",\n    \"amount\": \"$2200\",\n    \"bank\": \"Miami City Saving Union\",\n    \"branch\": \"Miami Centre Branch Highland and Avenue 123 Miami USA\"\n  }\n}",
        "{\n  \"document_type\": \"pan\",\n  \"named_entities\": {\n    \"name\": \"Amardeep Singh\",\n    \"father_name\": \"Surjit Singh\",\n    \"dob\": \"09/11/1995\",\n    \"pan_number\": \"LIWPS9203C\"\n  }\n}",
        "{\n  \"document_type\": \"credit card\",\n  \"named_entities\": {\n    \"name\": \"CF Frost\",\n    \"card_number\": \"3159 876543 21001\",\n    \"expiry_date\": \"10/28\",\n    \"issuer\": \"American Express\"\n  }\n}",
        "{\n  \"document_type\": \"aadhaar\",\n  \"named_entities\": {\n    \"name\": \"Aarav Kapoor\",\n    \"dob\": \"14/02/1990\",\n    \"gender\": \"Male\",\n    \"aadhaar_number\": \"7894 5623 1478\",\n    \"vid\": \"1234 5678 9012 3456\",\n    \"issue_date\": \"14/02/2020\"\n  }\n}",
        "{\n  \"document_type\": \"cheque\",\n  \"named_entities\": {\n    \"name\": \"Neha Sharma\",\n    \"address\": \"45 Elite Avenue Pune MH 411045\",\n    \"cheque_number\": \"B5678\",\n    \"date\": \"30/09/2023\",\n    \"payee\": \"Amardeep Singh\",\n    \"amount\": \"$1500\",\n    \"bank\": \"Pune City Cooperative Bank\",\n    \"branch\": \"Pune Elite Branch\"\n  }\n}",
        "{\n  \"document_type\": \"pan\",\n  \"named_entities\": {\n    \"name\": \"Jane Doe\",\n    \"father_name\": \"Michael Doe\",\n    \"dob\": \"12/08/1982\",\n    \"pan_number\": \"HGXPS3452K\"\n  }\n}",
        "{\n  \"document_type\": \"credit card\",\n  \"named_entities\": {\n    \"name\": \"Neha Sharma\",\n    \"card_number\": \"5678 4321 2109 8765\",\n    \"expiry_date\": \"07/25\",\n    \"issuer\": \"MasterCard\"\n  }\n}",
        "{\n  \"document_type\": \"cheque\",\n  \"named_entities\": {\n    \"name\": \"Michael Doe\",\n    \"address\": \"1234 Main St, New York, NY, 10001\",\n    \"cheque_number\": \"C9012\",\n    \"date\": \"01/12/2023\",\n    \"payee\": \"Aarav Kapoor\",\n    \"amount\": \"$5000\",\n    \"bank\": \"NY State Bank\",\n    \"branch\": \"NYC Branch\"\n  }\n}",
        "{\n  \"document_type\": \"aadhaar\",\n  \"named_entities\": {\n    \"name\": \"CF Frost\",\n    \"dob\": \"1980\",\n    \"gender\": \"Male\",\n    \"aadhaar_number\": \"6543 3210 9087\",\n    \"vid\": \"2109 8765 4321 1098\",\n    \"issue_date\": \"1985\"\n  }\n}"
    ]
}

# Convert to a Dataset object
dataset = Dataset.from_dict(data)

# Verify the dataset
print(dataset)

print(dataset[1:2])
dataset = dataset.map(formatting_prompts_func, batched = True,)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10
})
{'instruction': ['Identify the type of document and extract named entities such as name, email, ID, age, address, and others, if present.'], 'input': ['Jane Doe 123 Great Linclon Road Miami MA 11223 Check No. Date 26/06/2024 Pay to the order of Cairo Mohammed $2200 The sum of two thousand two hundred dollars Miami City Saving Union Miami Centre Branch Highland and Avenue 123 Miami USA Memo Authorized signature A1234 – 556789839292892 – 0123'], 'output': ['{\n  "document_type": "cheque",\n  "named_entities": {\n    "name": "Jane Doe",\n    "address": "123 Great Linclon Road Miami MA 11223",\n    "cheque_number": "A1234",\n    "date": "26/06/2024",\n    "payee": "Cairo Mohammed",\n    "amount": "$2200",\n    "bank": "Miami City Saving Union",\n    "branch": "Miami Centre Branch Highland and Avenue 123 Miami USA"\n  }\n}']}


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.004 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.178800
2,1.844200
3,2.042200
4,2.101700
5,1.735900
6,1.798100
7,1.380900
8,0.945500
9,0.886600
10,0.773400


In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

261.0302 seconds used for training.
4.35 minutes used for training.
Peak reserved memory = 6.723 GB.
Peak reserved memory for training = 0.719 GB.
Peak reserved memory % of max memory = 45.586 %.
Peak reserved memory for training % of max memory = 4.875 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [13]:

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Identify the type of document and extract named entities such as name, email, ID, age, address, and others, if present.", # instruction
        ", PUBLIC BANK IHOwG KLNT 2 119171 Domsttt 0in333r ipipincrneratiYm 87t Xleh Dulnuningatrilalnld Qarry BxY Nkaye7t nrpr GN, UntnkMll J04r1t2a Kik Kid /61 A James HII Moue X SIX HUNDRED AND NINETYNIN ETHOUSAND, FOUR HUNDRED AND TK HK ,699,422,81 WONG CHUNG YEE 0 a2zd9 02858 IYGo6 i03frPUBLIC BANK IHOwG KLNT 2 119171 Domsttt 0in333r ipipincrneratiYm 87t Xleh Dulnuningatrilalnld Qarry BxY Nkaye7t nrpr GN, UntnkMll J04r1t2a Kik Kid /61 A James HII Moue X SIX HUNDRED AND NINETYNIN ETHOUSAND, FOUR HUNDRED AND TK HK ,699,422,81 WONG CHUNG YEE 0 a2zd9 02858 IYGo6 i03fr", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nIdentify the type of document and extract named entities such as name, email, ID, age, address, and others, if present.\n\n### Input:\n, PUBLIC BANK IHOwG KLNT 2 119171 Domsttt 0in333r ipipincrneratiYm 87t Xleh Dulnuningatrilalnld Qarry BxY Nkaye7t nrpr GN, UntnkMll J04r1t2a Kik Kid /61 A James HII Moue X SIX HUNDRED AND NINETYNIN ETHOUSAND, FOUR HUNDRED AND TK HK,699,422,81 WONG CHUNG YEE 0 a2zd9 02858 IYGo6 i03frPUBLIC BANK IHOwG KLNT 2 119171 Domsttt 0in333r ipipincrneratiYm 87t Xleh Dulnuningatrilalnld Qarry BxY Nkaye7t nrpr GN, UntnkMll J04r1t2a Kik Kid /61 A James HII Moue X SIX HUNDRED AND NINETYNIN ETHOUSAND, FOUR HUNDRED AND TK HK,699,422,81 WONG CHUNG YEE 0 a2zd9 02858 IYGo6 i03fr\n\n### Response:\n{\n  "document_type": "cheque",\n  "named_entities": {\n    "name": "James HII",\n  

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [14]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [15]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
The Eiffel Tower, named after its designer, Gustave Eiffel, is a famous landmark located in Paris, France. It is a wrought iron tower that was built in 1889 as the entrance arch for the Exposition Universelle, a World’s Fair. The tower stands at a height of 324 meters (1,063 feet) and is a prominent symbol of Paris, known for its distinct shape and iconic status.

### Named Entities:
{
  "named_entities": {
    "name": "Eiffel Tower",
    "type": "tall tower",
    "location": "Paris, France"



You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [16]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [17]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.


* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.



In [18]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )